## Pipeline using Backward feature selection

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import warnings
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import Image
warnings.filterwarnings('ignore')

In [48]:
train_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("compete.csv")

In [49]:
train_df = train_df.drop(['is_host_login', 'num_outbound_cmds'], axis=1);
test_df = test_df.drop(['is_host_login', 'num_outbound_cmds'], axis=1);

In [50]:
train_df = pd.get_dummies(train_df, columns=['protocol_type'])
test_df = pd.get_dummies(test_df, columns=['protocol_type'])

In [51]:
from sklearn import preprocessing

cat_cols = ['service', 'flag']
for col in cat_cols:
    if col in train_df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(list(train_df[col].astype(str).values) + list(test_df[col].astype(str).values))
        train_df[col] = le.transform(list(train_df[col].astype(str).values))
        test_df[col] = le.transform(list(test_df[col].astype(str).values))   

In [52]:
numerical_features = list(train_df.columns[train_df.dtypes != object].values[:-1])
categorical_features = list(train_df.columns[train_df.dtypes == object].values)

corr_table = train_df.corr()
triu = corr_table.where(np.triu(np.ones(corr_table.shape) ,k=1).astype(np.bool))
to_drop = [feat for feat in triu.columns if any(triu[feat] > 0.95)]

train_df = train_df.drop(to_drop, axis=1)

for feat in to_drop:
    if feat in categorical_features:
        categorical_features.remove(feat)
    else:
        numerical_features.remove(feat)

print(f'\nFeatures dropped: {to_drop}')
# plt.figure(figsize=(50, 30))
# _ = sns.heatmap(corr_table, annot=True, fmt='.2f')


Features dropped: ['num_root', 'srv_serror_rate', 'srv_rerror_rate', 'dst_host_same_srv_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'protocol_type_icmp']


In [53]:
from sklearn.model_selection import train_test_split

X = train_df.drop('class', axis=1)
y = train_df['class']

### BACKWARD Feature Selection

In [54]:
import statsmodels.api as sm

X=sm.add_constant(X)
model=sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  class   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                 1.828e+05
Date:                Sun, 16 May 2021   Prob (F-statistic):               0.00
Time:                        14:13:33   Log-Likelihood:             3.2714e+05
No. Observations:              345815   AIC:                        -6.542e+05
Df Residuals:                  345782   BIC:                        -6.539e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.3456      0.009     40.554      0.000       0.329       0.362
duration                    -1.073e-05    2.7e-07    -39.760      0.000   -1.13e-05   -1.02e-05
service                        -0.0001   3.08e-05     -3.476      0.001      -0.000   -4.67e-05
flag                            0.0777      0.001     87.368      0.000       0.076       0.079
src_bytes                     1.76e-07   2.74e-09     64.159      0.000    1.71e-07    1.81e-07
dst_bytes                    1.394e-07   4.86e-09     28.708      0.000     1.3e-07    1.49e-07
land                            0.2644      0.024     11.168      0.000       0.218       0.311
wrong_fragment                  0.2846      0.001    226.823      0.000       0.282       0.287
urgent                          0.0075      0.026      0.282      0.778      -0.044       0.059
hot                             0.0728      0.000    191.997      0.000       0.072       0.074
num_failed_logins               0.2255      0.010     22.982      0.000       0.206       0.245
logged_in                       0.0708      0.003     24.258      0.000       0.065       0.077
num_compromised                 0.0034      0.000     23.157      0.000       0.003       0.004
root_shell                      0.4084      0.017     23.535      0.000       0.374       0.442
su_attempted                   -0.5903      0.027    -21.885      0.000      -0.643      -0.537
num_file_creations             -0.0011      0.002     -0.680      0.496      -0.004       0.002
num_shells                     -0.0231      0.014     -1.645      0.100      -0.051       0.004
num_access_files               -0.0601      0.005    -12.341      0.000      -0.070      -0.051
is_guest_login                 -0.9158      0.008   -114.949      0.000      -0.931      -0.900
count                        7.596e-05   4.85e-06     15.677      0.000    6.65e-05    8.55e-05
srv_count                       0.0010   7.06e-06    135.976      0.000       0.001       0.001
serror_rate                     0.5915      0.004    139.010      0.000       0.583       0.600
rerror_rate                     0.8696      0.007    122.166      0.000       0.856       0.884
same_srv_rate                  -0.6804      0.002   -290.782      0.000      -0.685      -0.676
diff_srv_rate                  -0.2738      0.003   -100.805      0.000      -0.279      -0.268
srv_diff_host_rate              0.0034      0.001      2.704      0.007       0.001       0.006
dst_host_count                  0.0005   4.22e-06    123.039      0.000       0.001       0.001
dst_host_srv_count             -0.0004   5.48e-06    -74.123      0.000      -0.000      -0.000
dst_host_diff_srv_rate          0.0327      0.003     11.813      0.000       0.027       0.038
dst_host_sa

### Remove the columns with P value greater than 0.05

In [55]:
X=X.drop(['urgent'],axis=1)
model=sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  class   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                 1.887e+05
Date:                Sun, 16 May 2021   Prob (F-statistic):               0.00
Time:                        14:13:34   Log-Likelihood:             3.2714e+05
No. Observations:              345815   AIC:                        -6.542e+05
Df Residuals:                  345783   BIC:                        -6.539e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.3456      0.009     40.563      0.000       0.329       0.362
duration                    -1.073e-05    2.7e-07    -39.760      0.000   -1.13e-05   -1.02e-05
service                        -0.0001   3.08e-05     -3.478      0.001      -0.000   -4.68e-05
flag                            0.0777      0.001     87.371      0.000       0.076       0.079
src_bytes                     1.76e-07   2.74e-09     64.159      0.000    1.71e-07    1.81e-07
dst_bytes                    1.395e-07   4.86e-09     28.716      0.000     1.3e-07    1.49e-07
land                            0.2644      0.024     11.168      0.000       0.218       0.311
wrong_fragment                  0.2846      0.001    226.823      0.000       0.282       0.287
hot                             0.0728      0.000    191.997      0.000       0.072       0.074
num_failed_logins               0.2260      0.010     23.387      0.000       0.207       0.245
logged_in                       0.0708      0.003     24.262      0.000       0.065       0.077
num_compromised                 0.0034      0.000     23.191      0.000       0.003       0.004
root_shell                      0.4084      0.017     23.534      0.000       0.374       0.442
su_attempted                   -0.5907      0.027    -21.929      0.000      -0.643      -0.538
num_file_creations             -0.0011      0.002     -0.677      0.498      -0.004       0.002
num_shells                     -0.0231      0.014     -1.645      0.100      -0.051       0.004
num_access_files               -0.0600      0.005    -12.338      0.000      -0.070      -0.050
is_guest_login                 -0.9158      0.008   -114.948      0.000      -0.931      -0.900
count                        7.595e-05   4.85e-06     15.676      0.000    6.65e-05    8.54e-05
srv_count                       0.0010   7.06e-06    135.978      0.000       0.001       0.001
serror_rate                     0.5914      0.004    139.010      0.000       0.583       0.600
rerror_rate                     0.8696      0.007    122.168      0.000       0.856       0.884
same_srv_rate                  -0.6804      0.002   -290.791      0.000      -0.685      -0.676
diff_srv_rate                  -0.2738      0.003   -100.816      0.000      -0.279      -0.269
srv_diff_host_rate              0.0034      0.001      2.703      0.007       0.001       0.006
dst_host_count                  0.0005   4.22e-06    123.041      0.000       0.001       0.001
dst_host_srv_count             -0.0004   5.48e-06    -74.124      0.000      -0.000      -0.000
dst_host_diff_srv_rate          0.0327      0.003     11.820      0.000       0.027       0.038
dst_host_same_src_port_rate     0.0820      0.002     48.458      0.000       0.079       0.085
dst_host_sr

In [56]:
X=X.drop(['num_file_creations'],axis=1)
model=sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  class   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                 1.950e+05
Date:                Sun, 16 May 2021   Prob (F-statistic):               0.00
Time:                        14:13:35   Log-Likelihood:             3.2714e+05
No. Observations:              345815   AIC:                        -6.542e+05
Df Residuals:                  345784   BIC:                        -6.539e+05
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.3456      0.009     40.566      0.000       0.329       0.362
duration                    -1.074e-05   2.69e-07    -39.922      0.000   -1.13e-05   -1.02e-05
service                        -0.0001   3.08e-05     -3.497      0.000      -0.000   -4.73e-05
flag                            0.0777      0.001     87.371      0.000       0.076       0.079
src_bytes                     1.76e-07   2.74e-09     64.169      0.000    1.71e-07    1.81e-07
dst_bytes                    1.395e-07   4.86e-09     28.714      0.000     1.3e-07    1.49e-07
land                            0.2644      0.024     11.169      0.000       0.218       0.311
wrong_fragment                  0.2846      0.001    226.825      0.000       0.282       0.287
hot                             0.0728      0.000    192.050      0.000       0.072       0.074
num_failed_logins               0.2260      0.010     23.385      0.000       0.207       0.245
logged_in                       0.0708      0.003     24.257      0.000       0.065       0.077
num_compromised                 0.0034      0.000     23.207      0.000       0.003       0.004
root_shell                      0.4078      0.017     23.530      0.000       0.374       0.442
su_attempted                   -0.5903      0.027    -21.919      0.000      -0.643      -0.537
num_shells                     -0.0236      0.014     -1.681      0.093      -0.051       0.004
num_access_files               -0.0604      0.005    -12.482      0.000      -0.070      -0.051
is_guest_login                 -0.9157      0.008   -114.954      0.000      -0.931      -0.900
count                        7.592e-05   4.84e-06     15.670      0.000    6.64e-05    8.54e-05
srv_count                       0.0010   7.06e-06    135.977      0.000       0.001       0.001
serror_rate                     0.5914      0.004    139.009      0.000       0.583       0.600
rerror_rate                     0.8696      0.007    122.167      0.000       0.856       0.884
same_srv_rate                  -0.6805      0.002   -290.876      0.000      -0.685      -0.676
diff_srv_rate                  -0.2739      0.003   -100.840      0.000      -0.279      -0.269
srv_diff_host_rate              0.0034      0.001      2.706      0.007       0.001       0.006
dst_host_count                  0.0005   4.22e-06    123.049      0.000       0.001       0.001
dst_host_srv_count             -0.0004   5.48e-06    -74.136      0.000      -0.000      -0.000
dst_host_diff_srv_rate          0.0328      0.003     11.844      0.000       0.027       0.038
dst_host_same_src_port_rate     0.0820      0.002     48.470      0.000       0.079       0.085
dst_host_srv_diff_host_rate     0.5339      0.005    103.368      0.000       0.524       0.544
protocol_ty

In [57]:
X=X.drop(['num_shells'],axis=1)
model=sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  class   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                 2.017e+05
Date:                Sun, 16 May 2021   Prob (F-statistic):               0.00
Time:                        14:13:37   Log-Likelihood:             3.2714e+05
No. Observations:              345815   AIC:                        -6.542e+05
Df Residuals:                  345785   BIC:                        -6.539e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.3456      0.009     40.567      0.000       0.329       0.362
duration                    -1.074e-05   2.69e-07    -39.908      0.000   -1.13e-05   -1.02e-05
service                        -0.0001   3.08e-05     -3.479      0.001      -0.000   -4.68e-05
flag                            0.0777      0.001     87.377      0.000       0.076       0.079
src_bytes                     1.76e-07   2.74e-09     64.179      0.000    1.71e-07    1.81e-07
dst_bytes                    1.395e-07   4.86e-09     28.728      0.000     1.3e-07    1.49e-07
land                            0.2645      0.024     11.172      0.000       0.218       0.311
wrong_fragment                  0.2846      0.001    226.836      0.000       0.282       0.287
hot                             0.0728      0.000    192.045      0.000       0.072       0.074
num_failed_logins               0.2262      0.010     23.401      0.000       0.207       0.245
logged_in                       0.0707      0.003     24.232      0.000       0.065       0.076
num_compromised                 0.0034      0.000     23.279      0.000       0.003       0.004
root_shell                      0.4020      0.017     23.661      0.000       0.369       0.435
su_attempted                   -0.5899      0.027    -21.904      0.000      -0.643      -0.537
num_access_files               -0.0605      0.005    -12.517      0.000      -0.070      -0.051
is_guest_login                 -0.9156      0.008   -114.944      0.000      -0.931      -0.900
count                        7.585e-05   4.84e-06     15.656      0.000    6.64e-05    8.53e-05
srv_count                       0.0010   7.06e-06    135.982      0.000       0.001       0.001
serror_rate                     0.5914      0.004    139.004      0.000       0.583       0.600
rerror_rate                     0.8696      0.007    122.166      0.000       0.856       0.884
same_srv_rate                  -0.6805      0.002   -290.957      0.000      -0.685      -0.676
diff_srv_rate                  -0.2739      0.003   -100.863      0.000      -0.279      -0.269
srv_diff_host_rate              0.0034      0.001      2.719      0.007       0.001       0.006
dst_host_count                  0.0005   4.22e-06    123.038      0.000       0.001       0.001
dst_host_srv_count             -0.0004   5.47e-06    -74.134      0.000      -0.000      -0.000
dst_host_diff_srv_rate          0.0329      0.003     11.891      0.000       0.027       0.038
dst_host_same_src_port_rate     0.0819      0.002     48.445      0.000       0.079       0.085
dst_host_srv_diff_host_rate     0.5339      0.005    103.369      0.000       0.524       0.544
protocol_type_tcp              -0.4106      0.004   -111.891      0.000      -0.418      -0.403
protocol_ty

In [58]:
X = X.drop(['const'], axis=1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [60]:
import lightgbm as lgb

clf = lgb.LGBMClassifier(bagging_fraction=0.8446682014322044, feature_fraction= 0.35294922284424557,
 learning_rate= 0.5683390979599586,
 max_bin= 21,
 max_depth= 28,
 min_data_in_leaf= 77,
 min_sum_hessian_in_leaf= 0.2693920974409014,
 num_leaves= 35,
 subsample= 0.2095972365619459,
 objective= 'binary',
 metric= 'auc',
 is_unbalance = True,
 boost_from_average = False)
clf.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.35294922284424557, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.35294922284424557
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.2693920974409014, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.2693920974409014
[LightGBM] [Warning] bagging_fraction is set=0.8446682014322044, subsample=0.2095972365619459 will be ignored. Current value: bagging_fraction=0.8446682014322044


LGBMClassifier(bagging_fraction=0.8446682014322044, boost_from_average=False,
               feature_fraction=0.35294922284424557, is_unbalance=True,
               learning_rate=0.5683390979599586, max_bin=21, max_depth=28,
               metric='auc', min_data_in_leaf=77,
               min_sum_hessian_in_leaf=0.2693920974409014, num_leaves=35,
               objective='binary', subsample=0.2095972365619459)

In [61]:
preds = clf.predict(X_test)

acc_lgbm = (preds == y_test).sum().astype(float) / len(preds)*100

print("LGBM Classifier prediction accuracy is: %3.2f" % (acc_lgbm))

LGBM Classifier prediction accuracy is: 99.98


In [62]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, preds)
roc_value

0.9996512180612531

In [63]:
test_df = test_df.drop(to_drop, axis=1)
test_id = test_df.Id.values
test_df = test_df.drop("Id", axis=1)

In [64]:
test_df = test_df.drop(['num_shells', 'num_file_creations', 'urgent'], axis=1)

In [66]:
preds = clf.predict(test_df)
preds_lgbm = clf.predict(test_df)

In [69]:
submit = pd.DataFrame({'Id': test_id, 'class':preds})
submit.to_csv('backward_lgbm.csv', index=False)